List of original code files incorporated into this notebook:
- bias_err_T2.py
- bias_corr.py
- cycles.py

List of files deemed outdated:
- bias_mean.py

List of plot files incorporated:
- landsea.pdf
- T2/daily_errors.pdf
- T2/annual_daily_model.pdf (i.e. annual, daily cycles based on Lomb-Scargle 'model' invocation)
- T2/bias_corr_bars.pdf
- T2/bias_corr_kde.pdf
- T2/bias_daily_max.pdf
- T2/bias_daily_min.pdf
- T2/bias_maps.pdf
- T2/bias_maps2.pdf
- T2/MAE_maps.pdf
- T2/seasonal_cycles.pdf
- T2/seasonal_errors.pdf

In [ ]:
import sys, re
sys.path.append('../../code/python')
import numpy as np
import pandas as pd
import helpers as hh
import matplotlib.pyplot as plt
from matplotlib.figure import SubplotParams
from astropy.stats import LombScargle
%matplotlib inline

Map = hh.basemap()

def plot(B, cols=None, clims=None, cbars='row'):
    if cols is not None:
        B = B.loc[:, :, cols]
    if clims is None:
        cl = pd.concat((B.min(1).min(), B.max(1).max()), 1)
    for k, b in B.iteritems():
        for j, x in enumerate(B.minor_axis):
            ax = plt.subplot(B.shape[0], B.shape[2], k * B.shape[2] + j + 1)
            D = pd.concat((sta.loc[B.major_axis, ('lon', 'lat')], b[x]), axis=1).dropna()
            if k == 0:
                ax.set_title(x)
            Map.scatter(D['lon'].as_matrix(), D['lat'].as_matrix(), c=D.iloc[:,-1], marker='o', latlon=True)
            Map.drawcoastlines()
            Map.drawparallels(range(-32, -28, 1), labels=[j==0, 0, 0, 0])
            Map.drawmeridians(range(-72, -69, 1), labels=[0, 0, 0, k==B.shape[0]-1])
            if cbars=='all':
                plt.colorbar()
            else:
                if clims is None:
                    plt.clim(cl[0][k], cl[1][k])
                else:
                    try:
                        plt.clim(clims[k][0], clims[k][1])
                    except TypeError:
                        plt.clim(-clims[k], clims[k])
                if j == B.shape[2] - 1:
                    bb = ax.get_position()
                    plt.colorbar(
                        cax=fig.add_axes([bb.x1 + 0.02, bb.y0, 0.02, bb.y1 - bb.y0]))

# helper to get the correct elevation data for each model 'name'
def dem(s):
    if s=='d03_orl': return s
    elif s[:3]=='d03': return 'd03_op'
    else: return s[:3]

## Land-Sea mask

In [ ]:
fig = plt.figure(figsize=(13, 3))
plot(pd.Panel({0:S['land_mask']}))

Linear interpolation of land (1) and sea (0) values to station locations on the various model grids.

## T2

In [ ]:
D = pd.HDFStore('../../data/tables/station_data.h5')

# CEAZAMet station location info
sta = D['sta']

# CEAZAMet station 2m temperature in K
T = hh.stationize(D['ta_c'].xs('prom', 1, 'aggr').drop(10, 1, 'elev')) + 273.15

S = pd.HDFStore('../../data/tables/LinearLinear.h5')

# model 2m temperature linearly interpolated to station location
Tm = S['T2n']

# temperature bias (model minus stations)
B = Tm.add( -T )

# mean bias
Bm = B.mean(1)

# elevations of station locations on respective model DEMs
Z = S['z'][[dem(z) for z in B.items]]
Z.columns = B.items

# elevation difference (true station elevation minus DEM)
dz = (sta['elev'] - Z.T).T

### Mean bias

In [ ]:
fig = plt.figure(
    figsize=(15, 10),
    subplotpars=SubplotParams(left=0.08, right=0.86, wspace=0.06, hspace=0.06))
plt.set_cmap('coolwarm')
x = ['d01', 'd02', 'd03_0_00', 'd03_0_12', 'd03_orl', 'fnl']
plot(pd.Panel({0:dz, 1:Bm, 2:Bm - 0.0065 * dz}), cols=x, clims=[1750,13,13])

**top**: elevation bias (true station elevation minus model DEM elevation interpolated to station location)  
**middle**: 2m temperature bias (model - station)  
**bottom**: temperature bias after correcting for a mean lapse rate of 6.5K / km

In [ ]:
fig = plt.figure(
    figsize=(15,15),
    subplotpars=SubplotParams(left=0.08, right=0.86, wspace=0.06, hspace=0.06))
plt.set_cmap('coolwarm')
x = ['d02', 'd03_orl', 'd03_0_00', 'd03_0_12']
plot(pd.Panel({0:dz, 1:Bm, 2:Bm - 0.0065 * dz}), cols=x, clims=[1150, 10, 10])

same as above but for selected domains only (less variance for same color scales)

**top**: elevation bias (true station elevation minus model DEM elevation interpolated to station location)  
**middle**: 2m temperature bias (model - station)  
**bottom**: temperature bias after correcting for a mean lapse rate of 6.5K / km

In [ ]:
x = ['fnl', 'd01', 'd02', 'd03_orl', 'd03_0_00']

fig = plt.figure(
    figsize=(15, 12),
    subplotpars=SubplotParams(left=0.08, right=0.86, wspace=0.06, hspace=0.06))
plt.set_cmap('coolwarm')

day = lambda x: x.groupby(lambda y: y.date).min()
dt = day(T) # daily min of station data
P = pd.Panel({
    0: day(Tm).add(-dt).mean(1), # daily min of model data minus daily min of station data
    1: day( Tm.add( -0.0065 * dz, 1) ).add(-dt).mean(1), # model data corrected by standard lapse rate
    2: day( Tm.add(-Bm, 1) ).add(-dt).mean(1) # model data with mean bias removed
})

plot(P, cols=x, clims=[10, 10, 10])

**top**: mean bias of daily **minimum** temperatures (model minus station)  
**middle**: mean bias after correction for mean lapse rate of 6.5K / km  
**bottom**: mean bias of daily minimum after removing mean bias (over all records)

In [ ]:
x = ['fnl', 'd01', 'd02', 'd03_orl', 'd03_0_00']

fig = plt.figure(
    figsize=(15, 12),
    subplotpars=SubplotParams(left=0.08, right=0.86, wspace=0.06, hspace=0.06))
plt.set_cmap('coolwarm')

day = lambda x: x.groupby(lambda y: y.date).max()
dt = day(T) # daily min of station data
P = pd.Panel(
    {
        0: day(Tm).add(-dt).mean(1), # daily min of model data minus daily min of station data
        1: day( Tm.add( -0.0065 * dz, 1) ).add(-dt).mean(1), # model data corrected by standard lapse rate
        2: day( Tm.add(-Bm, 1) ).add(-dt).mean(1) # model data with mean bias removed
    }
)

plot(P, cols=x, clims=[10, 10, 10])

**top**: mean bias of daily **maximum** temperatures (model minus station)  
**middle**: mean bias after correction for mean lapse rate of 6.5K / km  
**bottom**: mean bias of daily maximum after removing mean bias (over all records)

### Mean absolute error (MAE)

In [ ]:
x = ['fnl', 'd01', 'd02', 'd03_orl', 'd03_0_00', 'd03_0_12']

fig = plt.figure(
    figsize=(15, 10),
    subplotpars=SubplotParams(left=0.08, right=0.86, wspace=0.06, hspace=0.06))
plt.set_cmap('gnuplot')

P = pd.Panel({
    0: abs(B).mean(1), # mean absolute error (MAE)
    1: abs( B.add( -0.0065 * dz, 1 ) ).mean(1), # MAE after lapse rate adjustment
    2: abs( B.add(-Bm, 1) ).mean(1) # MAE after removal of mean bias
})

plot(P, cols=x)

**top**: mean absolute error (MAE) of 2m temperature (model/station)  
**middle**: MAE after correction for mean lapse rate of 6.5K / km  
**bottom**: MAE after removal of mean bias

## Cycles

In [ ]:
# This gives a good current approximation to the solar year length in seconds
def pow(d, T=np.timedelta64(1, 'Y').astype('timedelta64[s]'), return_period=False):
    f = lambda k: k.astype(float)
    try:
        c = d.dropna()
        t = np.array(c.index, dtype='datetime64[s]')
        n = np.arange(t[0], t[0]+T, dtype='datetime64[h]').astype('datetime64[s]')
        if (t[-1] - t[0]) < T / 4: return np.nan
        x = c.as_matrix()
        y = LombScargle(f(t), x).model(f(n), f(T)**-1)
        return pd.Timestamp(n[y.argmax()]).month if return_period else max(y) - min(y)
    except:
        return np.nan

In [ ]:
x = ['obs', 'fnl', 'd01', 'd02', 'd03_orl', 'd03_0_00']

fig = plt.figure(
    figsize=(15, 7),
    subplotpars=SubplotParams(left=.1, right=.96, bottom=.06, top=.92, wspace=.1, hspace=.1))
plt.set_cmap('gnuplot')

d = dict(Tm)
d['obs'] = T
d = pd.Panel(d)
std = d.groupby(d.major_axis.month).std()
std.major_axis = std.major_axis.astype('datetime64[M]')

P = pd.Panel({
    0: d.apply(pow, 1),
    1: std.mean(1),
    2: std.apply(pow, 1)
})

plot(P, cols=x, cbars='all')

**top**: amplitude of annual cycle in 2m temperatures for observations and models (peak-to-peak, $2 \hat{U}$)  
**middle**: mean monthly standard deviation of 2m temperatures  
**bottom**: amplitude of annual cycle (as above) of monthly standard deviation

**Note**: The annual cycle is computed from a Lomb-Scargle periodogram, as this is known to work reliably even for lengths of records of only a fraction of the period under investigation. The amplitude is computed from the model reconstruction of the astropy.stats.LombScargle algorithm, since the normalization of the Lomb-Scargle periodogram is not directly related to that of a Fourier transform.

In [ ]:
x = ['fnl', 'd01', 'd02', 'd03_orl', 'd03_0_00']

fig = plt.figure(
    figsize=(15, 6),
    subplotpars=SubplotParams(left=.1, right=.96, bottom=.06, top=.92, wspace=.1, hspace=.1))
plt.set_cmap('gnuplot')

std = B.groupby(B.major_axis.month).std()
std.major_axis = std.major_axis.astype('datetime64[M]')

P = pd.Panel({
    0: B.apply(pow, 1),
    1: std.apply(pow, 1)
})

plot(P, cols=x, cbars='all')

**top**: amplitude of annual cycle (see above) of 2m temperature **bias** (model minus station)  
**bottom**: amplitude of annual cycle of monthly standard deviation of temperature bias

In [ ]:
x = ['obs', 'fnl', 'd01', 'd02', 'd03_orl', 'd03_0_00']

fig = plt.figure(
    figsize=(15, 3),
    subplotpars=SubplotParams(left=.1, right=.96, bottom=.06, top=.92, wspace=.12, hspace=.1))
plt.set_cmap('hsv')

plot(pd.Panel({0: d.apply(pow, 1, return_period=True)}), cols=x, clims=[(1,12)])

**Phase** of annual cycle of 2m temperatures (as month 1-12), computed from Lomb-Scargle reconstrution

In [ ]:
x = ['obs', 'fnl', 'd01', 'd02', 'd03_orl', 'd03_0_00']

fig = plt.figure(
    figsize=(15, 6),
    subplotpars=SubplotParams(left=.1, right=.96, bottom=.06, top=.92, wspace=.1, hspace=.1))
plt.set_cmap('gnuplot')

P = pd.Panel({
    0: d.apply(pow, 1),
    1: d.apply(pow, 1, T=np.timedelta64(1, 'D').astype('timedelta64[s]')),
})

plot(P, cols=x)

**top**: amplitude of annual cycle (see above) of 2m temperatures  
**bottom**: amplitude of daily cycle, computed in the same way

## Kernel density estimates (KDE) of errors

In [ ]:
from scipy.stats import gaussian_kde

# compute values of 2x2 contingency table (a, b, c, d)
def abcd(df):
    m = df.notnull().astype(int)
    n = df.shape[0]
    a = np.where(m.sum(axis=1)==2)[0].shape[0]
    b,c = [np.where(m.diff(axis=1)[1]==i)[0].shape[0] for i in [1,-1]]
    d = n-a-b-c
    return a,b,c,d,n

x = np.linspace(-20,20,100)

P = [Tm, Tm.add(-0.0065 * dz, 1), Tm.add(-Bm, 1), Tm.add(-B.apply(lambda x:x.rolling('7D').mean(), 1))]

cols = np.array([['fnl','d01','d02'],['d03_orl','d03_0_00','d03_0_12'],['d03_0_00','d03_1_00','d03_4_00']])
fig,axs = plt.subplots(*cols.shape, figsize=(15, 10))
mae = np.zeros(np.r_[cols.shape,4])
cold = np.zeros_like(mae)
heat = np.zeros_like(mae)
for i, row in enumerate(cols):
    for j, s in enumerate(row):
        plt.sca(axs[i,j])
        axs[i,j].set_title(cols[i,j])

        for h,f in enumerate(P):
            y = f[s]-T
            fg = f[s].groupby(f[s].index.date)
            Tg = T.groupby(T.index.date)
            mae[i,j,h] = abs(y).mean().mean()
            k = gaussian_kde(y.stack().dropna())

            p = pd.Panel({0: Tg.min(), 1:fg.min()}).to_frame()
            a,b,c,d,n = abcd(p[p<hh.K])
            cold[i,j,h] = a/(a+b+c)

            p = pd.Panel({0: Tg.max(), 1:fg.max()}).to_frame()
            a,b,c,d,n = abcd(p[p>hh.K+30])
            heat[i,j,h] = a/(a+b+c)
            plt.plot(x,k(x))

        plt.grid()
        
for ax in axs.flatten():
    ax.set_ylim(0,0.2)
    ax.set_yticklabels([])

for ax in axs[:2,:].flatten():
    ax.set_xticklabels([])

In [ ]:
fig,axs = plt.subplots(*cols.shape, figsize=(15, 10))
cy = axs[0,0]._get_lines.prop_cycler
col = [next(cy)['color'] for i in range(4)]
x = np.arange(4)
for i in range(cols.shape[0]):
    for j in range(cols.shape[1]):
        axs[i,j].set_title(cols[i,j])
        plt.sca(axs[i,j])
        p = np.r_[mae[i:i+1,j],cold[i:i+1,j],heat[i:i+1,j]]
        plt.bar(x,p[0,:],color=col)
        axs[i,j].set_ylim((0,5.1))
        ax = axs[i,j].twinx()
        ax.bar(x+5,p[1,:],color=col)
        ax.bar(x+10,p[2,:],color=col)
        ax.set_ylim((0,0.81))
        axs[i,j].set_xticks([1.5,6.5,11.5])
        if i==2: axs[i,j].set_xticklabels(['MAE','TS < 0C', 'TS > 30C'])
        else: axs[i,j].set_xticklabels([])
        if j>0: axs[i,j].set_yticklabels([])
        if j<2: ax.set_yticklabels([])
        if i==1:
            if j==0: axs[1,0].set_ylabel('MAE')
            if j==2: ax.set_ylabel('threat score (TS)')

MAE and threat score <cite data-cite="2031184/BVKVAVZP"></cite> (page 263) of two scenarios:
1. Daily minimum below 0$^{\circ}$C ('< 0C')
1. Daily maximum above 30$^{\circ}$C ('> 30C')

**blue**: without correction  
**yellow**: with lapse rate 6.5K/km correction  
**green**: with mean bias removed  
**red**: with one-week running mean bias removed  

**Note**: Threat score of 1 is perfect, 0 is worst. Gives proportion of correct 'yes' forcasts of events after removing 'no' forcasts from consideration. 

In [ ]:
D.close()
S.close()

In [ ]:
import mapping as mp
import interpolation as intp
from netCDF4 import Dataset

g2 = Dataset('../../data/WRF/3d/grid_d03.nc')
Map = mp.basemap(g2)

n0 = Dataset('../../data/WRF/3d/d03_day0.nc')
x0 = intp.nc_interp(n0, 'T2', sta, method='linear', map=Map)
j = hh.tsplit(x0)

x0_1 = x0.iloc[:j,:]
x0_2 = x0.iloc[j:,:]

n4 = Dataset('../../data/WRF/3d/T2_4.nc')
x4 = intp.nc_interp(n4, 'T2', sta, method='linear', map=Map)
x4_1 = x4.iloc[:j,:]
x4_2 = x4.iloc[j:,:]

In [ ]:
fig = plt.figure(figsize=(12,8))
for i,x in enumerate([Tm['d03_orl'],x0_1,x0_2,x4_1,x4_2]):
    y = (x-T)**2
    y = y.groupby(y.index.hour).mean().mean(1)
    plt.plot(y,label=['orl','op+0/00','op+0/12','op+4/00','op+4/12'][i])
plt.legend()
plt.grid()
xt = plt.gca().set_xticks(range(0,24,2))

Mean error stratified by hour of the day for selected experiments.
#### Legend
**orl**: Orlando's domain
**op**: forcast domain 3
**+a/b**: a - forecast lead time in days, b - initialization hour of GFS forecast